# Testing 
(Mainly locally, I don't know a method to get python notebook on vader5)
## Checking whether the code interpreter can be called

In [40]:

import requests

base_url = 'http://localhost:8502'

auth_key = "qA94VhroHMHFN55inWgfAAkt1WEmzQ4J" # Only for testing
auth_string = "&auth_key=" + auth_key


user_input = "This is a test regarding your capabilities of using the code_interpreter tool. Please call the code_interpreter tool with the input \"test = 2+2\ntest\"."
url = base_url + '/streamresponse?input=' + user_input + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

# Note that this way to cunsume the response chunks it and those chunks are surprisingly small, cutting up the response.
# A better way would be `for delta in response.iter_content(chunk_size=1024):`
for delta in response:
    print(delta)
    complete_response.append(delta.decode("utf-8"))

b'{"variant":"ServerHint","content":"thread_id:zMgRy7runA9zo55sXSLLSYOVIdNR3Dck"}'
b'{"variant":"Code","content":["","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["{\\"","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["code","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["\\":\\"","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["test","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":[" =","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":[" ","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["2","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["+","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["2","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["\\\\","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["nt","call_C7KbEcL27YHaswCNyc0vXfli"]}'
b'{"variant":"Code","content":["est","call_C7KbEcL27YHaswC

In [37]:
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

print(reconstructed_response)

['{"variant":"ServerHint","content":"thread_id:Xdv6RsVaLZgU7i2T0bytAtV3rfsKHaax"}', '{"variant":"Code","content":["","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["{\\"","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["code","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["\\":\\"","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["2","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["+","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["2","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Code","content":["\\"}","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"CodeOutput","content":["4","call_ccAGxjW9anTN3yH7E0JYASSQ"]}', '{"variant":"Assistant","content":""}', '{"variant":"Assistant","content":"The"}', '{"variant":"Assistant","content":" result"}', '{"variant":"Assistant","content":" of"}', '{"variant":"Assistant","content":" the"}', '{"variant":"Assistant","content":" calculation"}

In [38]:
# Now we can parse the response as JSON
import json
response_json = [json.loads(x) for x in reconstructed_response]
for element in response_json:
    print(element)

{'variant': 'ServerHint', 'content': 'thread_id:Xdv6RsVaLZgU7i2T0bytAtV3rfsKHaax'}
{'variant': 'Code', 'content': ['', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['{"', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['code', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['":"', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['2', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['+', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['2', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Code', 'content': ['"}', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'CodeOutput', 'content': ['4', 'call_ccAGxjW9anTN3yH7E0JYASSQ']}
{'variant': 'Assistant', 'content': ''}
{'variant': 'Assistant', 'content': 'The'}
{'variant': 'Assistant', 'content': ' result'}
{'variant': 'Assistant', 'content': ' of'}
{'variant': 'Assistant', 'content': ' the'}
{'variant': 'Assistant', 'content': ' calculation

In [39]:
# We can use this to easily extract only the Assistant's responses
assistant_responses = [x for x in response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

The result of the calculation \(2 + 2\) is \(4\). If you have any other tests or questions regarding climate data analysis, feel free to ask!


In [18]:
# Testing the get_thread endpoint
thread_id = "dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw" # for testing
url = base_url + '/getthread?thread_id=' + thread_id + auth_string
response = requests.get(url)
print(response.json())

[{'variant': 'ServerHint', 'content': 'thread_id:dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw'}, {'variant': 'User', 'content': 'This is a test regarding your capabilities of using the code_interpreter tool. Please call the code_interpreter tool with the input \\"2+2\\".'}, {'variant': 'Code', 'content': ''}, {'variant': 'Code', 'content': '{\\'}, {'variant': 'Code', 'content': 'code'}, {'variant': 'Code', 'content': '\\":\\'}, {'variant': 'Code', 'content': '2'}, {'variant': 'Code', 'content': '+'}, {'variant': 'Code', 'content': '2'}, {'variant': 'Code', 'content': '\\"}'}, {'variant': 'CodeOutput', 'content': 'The code interpreter was successfully called, but is not yet implemented. The inputs to this function were: code_interpreter ; Some(\\"{\\\\\\"code\\\\\\":\\\\\\"2+2\\\\\\"}\\"). @FrevaGPT, you may tell the user that the tool call was called and interpreted, but not yet implemented.'}, {'variant': 'OpenAIError', 'content': 'Error getting response.'}, {'variant': 'StreamEnd', 'content': 'S